### References

<ul>
    <li>Trending twitter<br></li>
    https://twitter-trends.iamrohit.in/
    <li>Removing accented characters<br></li>
    https://stackoverflow.com/a/2633310/13557629
    <li>Importing package from filepath</li>
    https://stackoverflow.com/a/50395128/13557629
</ul>

In [1]:
import ipyparams
import regex as re
import numpy as np 
import pandas as pd
from importlib import reload
import connection
from datetime import datetime
import utils
import logging

<IPython.core.display.Javascript object>

In [2]:
utils.setup_logger(
    file_name='updating-extraction', 
    desc='Preparing extraction code for packaging. Moving as much of notebook code to modules as possible and documenting for ease of use for future users'
)

In [17]:
reload(connection)

<module 'connection' from '/home/rimov/Documents/Code/NLP/lin-que-dropping/src/twitter_connection/connection.py'>

In [2]:
gen_conf = utils.get_config()
conf = utils.get_config('e')

### Extraction

In [3]:
# TODO: key_name not working properly; python-decouple not finding .env file
#   potentially because it's looking in a different directory
es_conn = connection.TwitterConnection(
    lang='es',
    is_archive=True,
    key_name='riccelli'
)

# pt_conn = connection.TwitterConnection() 

The passed key name was not found: ('riccelli_key not found. Declare it as envvar or define a default value.',)
Traceback (most recent call last):
  File "/home/rimov/Documents/Code/NLP/lin-que-dropping/src/twitter_connection/connection.py", line 59, in auth
    token = key if key is not None else config(env_key_name)
  File "/home/rimov/anaconda3/envs/que_drop/lib/python3.10/site-packages/decouple.py", line 245, in __call__
    return self.config(*args, **kwargs)
  File "/home/rimov/anaconda3/envs/que_drop/lib/python3.10/site-packages/decouple.py", line 107, in __call__
    return self.get(*args, **kwargs)
  File "/home/rimov/anaconda3/envs/que_drop/lib/python3.10/site-packages/decouple.py", line 92, in get
    raise UndefinedValueError('{} not found. Declare it as envvar or define a default value.'.format(option))
decouple.UndefinedValueError: riccelli_key not found. Declare it as envvar or define a default value.


riccelli_key


In [19]:
conjugations = pd.read_csv(utils.get_project_root()/'data'/'twitter-conjugations-query.csv')
conjugations = conjugations.to_dict('records')

In [6]:
# List of verbs that shouldn't be pulled
finished = []

In [7]:
time = utils.get_str_datetime_now(True, True)

In [12]:
save_path = utils.make_dir(
    utils.get_save_path('e', 'twitter', False, 'es'), time 
)

for i, topic in enumerate(conjugations):
    # Skip verbs that were already pulled or are not desired
    if topic['verb'] in finished:
        continue
    
    pages = es_conn.paginate(save_path, 
                             (topic['verb'], topic['indicativo']), 
                             batch_size=500,
                             batch_num=20,
                             sleep_sec=1)
    
    print(f'Retrieved total {pages} pages for {topic["verb"]}.')
    
    finished.append(topic['verb'])

Retrieved total 4 pages for mencionar.
Retrieved total 6 pages for gritar.
Retrieved total 0 pages for suspirar.
Retrieved total 4 pages for reclamar.
Retrieved total 20 pages for contar.
Retrieved total 20 pages for suponer.
Retrieved total 20 pages for saber.
Retrieved total 20 pages for pensar.
Retrieved total 17 pages for imaginar.
Retrieved total 20 pages for dudar.
Retrieved total 20 pages for creer.
Retrieved total 20 pages for recordar.
Retrieved total 20 pages for acordar.
Retrieved total 2 pages for temer.
Retrieved total 8 pages for recomendar.
Retrieved total 20 pages for parecer.
Retrieved total 20 pages for entender.
Retrieved total 5 pages for negar.
Retrieved total 4 pages for apostar.
Retrieved total 0 pages for predecir.
Retrieved total 1 pages for prever.
Retrieved total 20 pages for sentir.
Retrieved total 2 pages for comprobar.
Retrieved total 1 pages for adivinar.
Retrieved total 3 pages for lamentar.
Retrieved total 1 pages for rogar.
Retrieved total 20 pages for